# NTBD Projeto
## FI 02 - Web Scraping do Site TudoGostoso
#### Grupo 01
    - Felipe Ottoni Pereira
    - Letícia Almeida Paulino de Alencar Ferreira

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import re


# Configurações do navegador
options = Options()
#options.add_argument("--headless")  # Precisamos da interface gráfica
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Caminho do ChromeDriver
chromedriver_path = r"C:\Users\otton\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"  
driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)

try:
    # Abre o navegador na página da receita
    driver.get("https://www.tudogostoso.com.br/receita/62031-fricasse-de-frango-sem-milho.html")

    # Carregamento da página principal
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        print("Página carregada com sucesso.")
    except Exception as e:
        print("A página não carregou corretamente ou demorou muito.")
        

    # Verificar se há o botão "Ir para o site"
    try:
        botao_ir = driver.find_element(By.XPATH, r"//button[contains(text(), 'Ir para o site')]")
        botao_ir.click()
        print("Botão 'Ir para o site' clicado.")
        time.sleep(2)  # Esperar carregar após clicar
    except Exception:
        print("Botão 'Ir para o site' não apareceu.")

    print('-' * 30)

    # Captura o HTML atualizado
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")

    # Extrair informações da receita
    titulo = soup.find("span", class_="u-title-page u-align-center")
    titulo = titulo.text.strip() if titulo else "Nulo"

    print(titulo)

    rating_element = soup.find("span", class_="rating-grade")
    print(rating_element)
    rating = (
        float(re.search(r"\d+\.\d+|\d+", rating_element.text).group())
        if rating_element
        else "Nulo"
    )
    print(rating)

    votes_element = soup.find("span", class_="rating-votes")
    votes_text = votes_element.text.strip() if votes_element else "Nulo"
    votes = (
        float(re.search(r"\d[\d\.]*", votes_text).group())
        if votes_text != "Nulo"
        else "Nulo"
    )

    # Autor
    autor_tag = soup.select_one("div.is-11.is-8-tablet.u-color-primary a")
    # Extrai o texto e remove "Por "
    if autor_tag:
        autor_nome = autor_tag.text.strip().replace("Por ", "")

    # Número de porções
    portions = "Nulo"
    ingredients_section = soup.select_one(
        ".recipe-section.recipe-ingredients > header > h2"
    )
    if ingredients_section:
        portions_text = ingredients_section.text.strip()
        portions_match = re.search(r"\((\d+)\s+por", portions_text)
        if portions_match:
            portions = float(portions_match.group(1))

    # Tempo de preparo, dificuldade e custo
    prep_time, difficulty, cost = "Nulo", "Nulo", "Nulo"
    recipe_blocks = soup.select(".recipe-info")
    for recipe in recipe_blocks:
        info_items = recipe.select(".recipe-info-item")
        if len(info_items) > 0:
            prep_time = info_items[0].text.strip()
            hours = int(re.search(r"(\d+)h", prep_time).group(1)) if "h" in prep_time else 0
            minutes = int(re.search(r"(\d+)min", prep_time).group(1)) if "min" in prep_time else 0
            prep_time = hours * 60 + minutes
        if len(info_items) > 1:
            difficulty = info_items[1].text.strip()
        if len(info_items) > 2:
            cost = info_items[2].text.strip()

    # Armazena os dados
    receita = {
        "Título": titulo,
        "Autor": autor_nome,
        "Nota": rating,
        "Número de avaliações": votes,
        "Número de Porções": portions,
        "Tempo de Preparo (min)": prep_time,
        "Dificuldade": difficulty,
        "Custo": cost,
    }

    print(receita)
    
except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    # Fechar o navegador
    driver.quit()



Página carregada com sucesso.
Botão 'Ir para o site' não apareceu.
------------------------------
Fricassê de frango sem milho
<span class="rating-grade"><span class="u-bold">5</span> / 5
        </span>
5.0
{'Título': 'Fricassê de frango sem milho', 'Autor': 'Bárbara Goulart', 'Nota': 5.0, 'Número de avaliações': 9.0, 'Número de Porções': 6.0, 'Tempo de Preparo (min)': 40, 'Dificuldade': 'Nulo', 'Custo': 'Nulo'}


In [114]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import re


# Configurações do navegador
options = Options()
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

#num_pag = list(range(0, 1))  # Gera uma lista de 1 a 118
#https://www.tudogostoso.com.br/categorias/1002-recheios-e-coberturas?page={str(num_pag)}
#for i in num_pag:
    # Caminho do ChromeDriver
chromedriver_path = r"C:\Users\otton\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)

# URL da subcategoria
subcategoria_url = f"https://www.tudogostoso.com.br/categorias/1007-carnes-exoticas" #https://www.tudogostoso.com.br/categorias/1070-bolo-de-banana
categoria_fixa = "Carnes"
subcategoria_fixa = "Carnes exóticas"

try:
    # Acessar a página da subcategoria
    driver.get(subcategoria_url)

    # Espera até que os cards das receitas carreguem
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME, "card-listing"))
    )

    # Capturar os links das receitas na subcategoria
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")

    #print(soup.text)

    receita_links = [
        a["href"]
        for a in soup.select("div.grid.card-listing a.card-link")
        if "href" in a.attrs and "/receita/" in a["href"]
    ]

    print(f"Links das receitas encontrados: {len(receita_links)}")
    for link in receita_links:
        print(link)

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    driver.quit()

Links das receitas encontrados: 15
https://www.tudogostoso.com.br/receita/4331-lagarto-vapt-vupt.html
https://www.tudogostoso.com.br/receita/19766-coelho-assado-ao-molho-de-laranja.html
https://www.tudogostoso.com.br/receita/80690-jacare-ao-molho.html
https://www.tudogostoso.com.br/receita/248-carne-de-jacare.html
https://www.tudogostoso.com.br/receita/94999-escargot.html
https://www.tudogostoso.com.br/receita/81496-javali-ao-hidromel.html
https://www.tudogostoso.com.br/receita/2781-javali.html
https://www.tudogostoso.com.br/receita/46140-coelho-frito.html
https://www.tudogostoso.com.br/receita/125199-carne-de-capivara.html
https://www.tudogostoso.com.br/receita/6452-lombo-na-cerveja.html
https://www.tudogostoso.com.br/receita/139135-ra-ao-forno-ao-perfume-de-coco.html
https://www.tudogostoso.com.br/receita/141486-manicoba-paraense.html
https://www.tudogostoso.com.br/receita/79542-coelho-assado-no-forno.html
https://www.tudogostoso.com.br/receita/139160-paca-assada.html
https://www.tud

In [115]:
# Configurações do navegador
options = Options()
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

receitas = []
for link in receita_links:
        
    chromedriver_path = r"C:\Users\otton\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
    driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)
    
    try:
        receita_url = f"{link}"
        driver.get(receita_url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        print(f"Capturando: {receita_url}")

        # Captura o HTML atualizado
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")

        # Extrair informações da receita
        titulo = soup.find("span", class_="u-title-page u-align-center")
        titulo = titulo.text.strip() if titulo else "Nulo"

        rating_element = soup.find("span", class_="rating-grade")
        rating = (
            float(re.search(r"\d+\.\d+|\d+", rating_element.text).group())
            if rating_element
            else "Nulo"
        )

        votes_element = soup.find("span", class_="rating-votes")
        votes_text = votes_element.text.strip() if votes_element else "Nulo"
        votes = (
            float(re.search(r"\d[\d\.]*", votes_text).group())
            if votes_text != "Nulo"
            else "Nulo"
        )

        # Autor
        autor_tag = soup.select_one("div.is-11.is-8-tablet.u-color-primary a")
        # Extrai o texto e remove "Por "
        if autor_tag:
            autor_nome = autor_tag.text.strip().replace("Por ", "")

        # Número de porções
        portions = "Nulo"
        ingredients_section = soup.select_one(
            ".recipe-section.recipe-ingredients > header > h2"
        )
        if ingredients_section:
            portions_text = ingredients_section.text.strip()
            portions_match = re.search(r"\((\d+)\s+por", portions_text)
            if portions_match:
                portions = float(portions_match.group(1))

        # Tempo de preparo, dificuldade e custo
        prep_time, difficulty, cost = "Nulo", "Nulo", "Nulo"
        recipe_blocks = soup.select(".recipe-info")
        for recipe in recipe_blocks:
            info_items = recipe.select(".recipe-info-item")
            if len(info_items) > 0:
                prep_time = info_items[0].text.strip()
                hours = int(re.search(r"(\d+)h", prep_time).group(1)) if "h" in prep_time else 0
                minutes = int(re.search(r"(\d+)min", prep_time).group(1)) if "min" in prep_time else 0
                prep_time = hours * 60 + minutes
            if len(info_items) > 1:
                difficulty = info_items[1].text.strip()
            if len(info_items) > 2:
                cost = info_items[2].text.strip()

        # Armazena os dados
        receita = {
            "Título": titulo,
            "Autor": autor_nome,
            "Nota": rating,
            "Número de avaliações": votes,
            "Categoria": categoria_fixa,
            "Subcategoria": subcategoria_fixa,
            "Número de Porções": portions,
            "Tempo de Preparo (min)": prep_time,
            "Dificuldade": difficulty,
            "Custo": cost,
            "URL": receita_url,
        }

        receitas.append(receita)
        time.sleep(3)

    except Exception as e:
        print(f"Erro ao processar receita: {link}. Erro: {e}")
        time.sleep(3)

    finally:
        driver.quit()

# Exibir as receitas coletadas
for r in receitas:
    print(r)
    print("-" * 50)

Capturando: https://www.tudogostoso.com.br/receita/4331-lagarto-vapt-vupt.html
Capturando: https://www.tudogostoso.com.br/receita/19766-coelho-assado-ao-molho-de-laranja.html
Capturando: https://www.tudogostoso.com.br/receita/80690-jacare-ao-molho.html
Capturando: https://www.tudogostoso.com.br/receita/248-carne-de-jacare.html
Capturando: https://www.tudogostoso.com.br/receita/94999-escargot.html
Capturando: https://www.tudogostoso.com.br/receita/81496-javali-ao-hidromel.html
Capturando: https://www.tudogostoso.com.br/receita/2781-javali.html
Capturando: https://www.tudogostoso.com.br/receita/46140-coelho-frito.html
Capturando: https://www.tudogostoso.com.br/receita/125199-carne-de-capivara.html
Capturando: https://www.tudogostoso.com.br/receita/6452-lombo-na-cerveja.html
Capturando: https://www.tudogostoso.com.br/receita/139135-ra-ao-forno-ao-perfume-de-coco.html
Capturando: https://www.tudogostoso.com.br/receita/141486-manicoba-paraense.html
Capturando: https://www.tudogostoso.com.br

In [ ]:
import psycopg2

# Inserir dados no banco de dados
try:
    conn = psycopg2.connect(
        host="localhost",  
        port="5432",  # porta padrão do PostgreSQL
        database="aux_receitas_dw",
        user="postgres",
        password="labbd"
    ) 
    cursor = conn.cursor()

    insert_query = """
        INSERT INTO receitas (nome, autor, nota, numero_avaliacoes, categoria, subcategoria, numero_porcoes, tempo_preparo_min, dificuldade, custo)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    for r in receitas:
        cursor.execute(insert_query, (
            r["Título"],
            r["Autor"],
            r["Nota"],
            r["Número de avaliações"],
            r["Categoria"],
            r["Subcategoria"],
            str(r["Número de Porções"]),
            str(r["Tempo de Preparo (min)"]),
            r["Dificuldade"],
            r["Custo"],
        ))
        #cursor.execute(insert_query, (titulo, rating, votes_number,category, subcategory, portions, prep_time, difficulty, cost))
        conn.commit()
        print("Receita inserida com sucesso!")

except (Exception, psycopg2.DatabaseError) as error:
    print(f"Erro ao inserir dados: {error}")


cursor.close()
conn.close()

Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!
Receita inserida com sucesso!


# inserções individuais: caso tenha dado errado

Inserção de receita unitária uma a uma por link, caso falhe a extração por subcategoria acima

In [1]:

def coletar_dados_receita(receita_url, categoria_fixa, subcategoria_fixa):
    """Coleta os dados da receita a partir da URL e retorna um dicionário com as informações."""
    
    # Configurações do navegador
    options = Options()
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    chromedriver_path = r"C:\Users\otton\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
    driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)

    try:
        driver.get(receita_url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        print(f"Capturando: {receita_url}")

        # Captura o HTML atualizado
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")

        # Extrair informações da receita
        titulo = soup.find("span", class_="u-title-page u-align-center")
        titulo = titulo.text.strip() if titulo else None

        rating_element = soup.find("span", class_="rating-grade")
        rating = (
            float(re.search(r"\d+\.\d+|\d+", rating_element.text).group())
            if rating_element else None
        )

        votes_element = soup.find("span", class_="rating-votes")
        votes_text = votes_element.text.strip() if votes_element else None
        votes = (
            float(re.search(r"\d[\d\.]*", votes_text).group())
            if votes_text else None
        )

        # Autor
        autor_tag = soup.select_one("div.is-11.is-8-tablet.u-color-primary a")
        autor_nome = autor_tag.text.strip().replace("Por ", "") if autor_tag else None

        # Número de porções
        portions = None
        ingredients_section = soup.select_one(".recipe-section.recipe-ingredients > header > h2")
        if ingredients_section:
            portions_text = ingredients_section.text.strip()
            portions_match = re.search(r"\((\d+)\s+por", portions_text)
            if portions_match:
                portions = float(portions_match.group(1))

        # Tempo de preparo, dificuldade e custo
        prep_time, difficulty, cost = "Nulo", "Nulo", "Nulo"
        recipe_blocks = soup.select(".recipe-info")
        for recipe in recipe_blocks:
            info_items = recipe.select(".recipe-info-item")
            if len(info_items) > 0:
                prep_time = info_items[0].text.strip()
                hours = int(re.search(r"(\d+)h", prep_time).group(1)) if "h" in prep_time else 0
                minutes = int(re.search(r"(\d+)min", prep_time).group(1)) if "min" in prep_time else 0
                prep_time = hours * 60 + minutes
            if len(info_items) > 1:
                difficulty = info_items[1].text.strip()
            if len(info_items) > 2:
                cost = info_items[2].text.strip()

        driver.quit()

        # Validação: só retorna os dados se tiver pelo menos título e nota
        if titulo and rating is not None:
            return {
                "Título": titulo,
                "Autor": autor_nome,
                "Nota": rating,
                "Número de avaliações": votes,
                "Categoria": categoria_fixa,
                "Subcategoria": subcategoria_fixa,
                "Número de Porções": portions,
                "Tempo de Preparo (min)": prep_time,
                "Dificuldade": difficulty,
                "Custo": cost,
            }
        else:
            print(f"⚠ Dados insuficientes para inserir no banco. Receita ignorada: {receita_url}")
            return None

    except Exception as e:
        print(f"⚠ Erro ao processar a receita: {receita_url}. Erro: {e}")
        driver.quit()
        return None


def inserir_receita_no_banco(receita):
    """Insere os dados da receita no banco de dados apenas se a coleta foi bem-sucedida."""
    
    if not receita:
        print("Receita não coletada corretamente. Nenhum dado será inserido.")
        return

    try:
        conn = psycopg2.connect(
            host="localhost",
            port="5432",
            database="aux_receitas_dw",
            user="postgres",
            password="labbd"
        )
        cursor = conn.cursor()

        insert_query = """
            INSERT INTO receitas (nome, autor, nota, numero_avaliacoes, categoria, subcategoria, numero_porcoes, tempo_preparo_min, dificuldade, custo)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (
            receita["Título"],
            receita["Autor"],
            receita["Nota"],
            receita["Número de avaliações"],
            receita["Categoria"],
            receita["Subcategoria"],
            str(receita["Número de Porções"]) if receita["Número de Porções"] else None,
            str(receita["Tempo de Preparo (min)"]) if receita["Tempo de Preparo (min)"] else None,
            receita["Dificuldade"],
            receita["Custo"],
        ))
        conn.commit()
        print("Receita inserida com sucesso!")

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao inserir no banco de dados: {error}")

    finally:
        cursor.close()
        conn.close()


# Exemplo:
link_da_receita = "https://www.tudogostoso.com.br/receita/85922-cuscuz-paulista-da-mama.html"
categoria = "Prato único"
subcategoria = "Cuscuz"

# Coletar dados
receita_coletada = coletar_dados_receita(link_da_receita, categoria, subcategoria)
print(receita_coletada)


NameError: name 'Options' is not defined

In [ ]:
# Inserir no banco apenas se os dados forem válidos
inserir_receita_no_banco(receita_coletada)

✅ Receita inserida com sucesso!
